## 无框架多层神经网络识别猫咪图片

此项目目的在熟练掌握多层全连接网络的基本原理。 使用python/numpy手工实现多层神经网络。 数据集使用两组h5格式的图片数据集。
需要导入的库如下：

In [1]:
import numpy as np
import h5py    # 用于导入h5数据
import matplotlib.pyplot as plt

np.random.seed(121)

### 1. 多层神经网络每一层输入输出维度关系
由于需要从头开始实现神经网络， 在争相传播和反向传播时， 需要矩阵运算维度保持匹配。 在开工支之前先把维度弄清楚。
比如输入层X维度是$(n^{[1]}=12288,m^{[1]}=209)$, 也就是特征有12288维， 样品数209个。具体维度如下：


<table style="width:100%">
	<tr>
		<th>网络层数</th>
		<th>W维度</th>
		<th>b维度</th>
		<th>Z值</th>
		<th>A维度</th>
	</tr>
	<tr>
		<td>Layer 1</td>
		<td>$(n^{[1]}, 12288)$</td>
		<td>$(n^{[1]}, 1)$</td>
		<td>$Z^{[1]}=W^{[1]}X + b^{[1]}$</td>
		<td>$(n^{[1]}, 209)$</td>
	</tr>

	<tr>
		<td>Layer 2</td>
		<td>$(n^{[2]}, n^{[1]})$</td>
		<td>$(n^{[2]}, 1)$</td>
		<td>$Z^{[2]}=W^{[2]}A^{[1]}+b^{[2]}$</td>
		<td>$(n^{[2]}, 209)$</td>
	</tr>

	<tr>
        	<td>$\vdots$</td> 
        	<td>$\vdots$</td> 
        	<td>$\vdots$</td> 
        	<td>$\vdots$</td> 
        	<td>$\vdots$</td> 
	</tr>

	<tr>
		<td>Layer L-1</td>
		<td>$(n^{[L-1]}, n^{[L-2]})$</td>
		<td>$(n^{[L-1]}, 1)$</td>
		<td>$Z^{[L-1]}=W^{[L-1]}A^{[L-2]} + b^{[L-2]}$</td>
		<td>$(n^{[L-1]}, 209)$</td>
	</tr>
	
	<tr>
		<td>Layer L</td>
		<td>$(n^{[L]}, n^{[L-1]})$</td>
		<td>$(n^{[L]}, 1)$</td>
		<td>$Z^{[L]}=W^{[L-1]}A^{[L-1]} + b^{[L-1]}$</td>
		<td>$(n^{[L]}, 209)$</td>
	</tr>
	
</table>

网络基本构成如下：
[Linear --> RELU](L-1层) --> [Linear --> Sigmoid]

### 4. 向后传播（Backward Propagation）
向后传播涉及一系列链式求导。中间L-1层的一系列梯度如下。 

激活函数的梯度

$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$

线性函数梯度
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$

而第L层的dAL如下：
$$dAL=\frac{\partial{J}}{\partial{A^{[L]}}}=-(\frac{Y}{A^{[L]}}-\frac{1-Y}{1-A^{[L]}})$$


#### 4.1 激活函数梯度
为了后面计算起来方便，先来写激活函数的梯度
**Relu**

### 3. 权值初始化
* 对于神经网络而言， 不能把多神经元的W全部初始化为0， 如果的这样的话， 训练出来每个神经元结果完全一样。 效果等同于一个逻辑回归函数。 
* 为了保证多层变换以后不至于输出结果过大导致梯度爆炸，初始权值尽量小一些。 使用np.random.randn(shape)*0.01初始化W
* b可以初始位0， np.zeros(shape)

In [2]:
def l_layer_init(layer_dims_list):
    """
    
    :param 
        list_layer_dims -- python list of dims of each layer. Each element is a int number of hidden unit
    :return: 
        param_weights: python dict of W,b for each layers
    """
    L = len(layer_dims_list)
    param_weights = {}
    np.random.seed(121)
    for l in range(1, L):
        W = np.random.randn(layer_dims_list[l],layer_dims_list[l-1]) * 0.01
        b = np.zeros([layer_dims_list[l], 1])
        param_weights['W'+str(l)] = W
        param_weights['b'+str(l)] = b
    return param_weights      

假设初始一个3层网络， 输入特征10维，第二层，3个hidden unit, 输出层1维。 隐藏层和输出层的参数权重维度应该是
W1(3,10), b1(3,1), W2(1, 3), b2(1, 1), 测试一下上述函数。 

In [3]:
test_dims_list = [10, 3, 1]
test_params_weights = l_layer_init(test_dims_list)
print('W1')
print(test_params_weights['W1'].shape)
print('b1')
print(test_params_weights['b1'].shape)
print('\nW2')
print(test_params_weights['W2'].shape)
print('b2')
print(test_params_weights['b2'].shape)

W1
(3, 10)
b1
(3, 1)

W2
(1, 3)
b2
(1, 1)


### 2. 向前传播Forward Propagation
向前传播包括两部分变换
* 线性变换
* 非线性激活函数变换
因此需要把需要的激活函数定义出来。 这个项目中， 隐藏层使用relu函数， 输出层牵扯到分类问题，使用sigmoid函数。

In [19]:
def relu(Z):
    cache = Z
    A = np.maximum(0, Z)
    return A, cache

def sigmoid(Z):
    cache = Z
    A = 1/(1+np.exp(-Z))
    return A, cache

激活函数定义完毕后，准备定义forward_propagation函数
#### 2.2 定义向前传播函数
* 第一层网络， $A^{[0]}=X$
* 前L-1层网络，$Z^{[l]}=W^{[l]}A^{[l-1]} + b^{[l]}，A^{[l]}=Relu(Z^{[l]})$
* 第L层网络， $Z^{[L]}=W^{[L]}A^{[L-1]} + b^{[L]}，A^{[L]}=sigmoid(Z^{[L]})$

In [20]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache


def forward_propagation(X, param_weights):
    """
    :param param_weights: python dict of parameters weights
    :param X: input matrix
    
    :return: 
    AL -- output layer activation value
    caches -- list of caches containing Z value for each layer
    """
    caches = []
    A = X
    L = len(param_weights)//2
    
    ## impelment [linear --> relu]*(L-1) --> [linear --> sigmoid]
    
    ## L-1 layers
    for l in range(1, L):
        A_prev = A
        Wl, bl = param_weights['W' + str(l)], param_weights['b'+str(l)]
        A, cache = linear_activation_forward(A_prev, Wl, bl, 'relu')
        caches.append(cache)
    WL, bL = param_weights['W' + str(L)], param_weights['b' + str(L)]
    AL, cache = linear_activation_forward(A, WL, bL, 'sigmoid')
    caches.append(cache)
    return AL, caches

### 3. Cost 函数
由于是二分类问题， 因此使用交叉熵来描述损失函数

$J(W,b)=\frac{1}{m}\sum_{i=1}^{m}((y^{(i)}log(a^{[L](i)})+(1-y^{(i)})log(1-a^{[L](i)}))$

In [7]:
def cost(AL, Y):
    """
    :param AL: output layer activation value
    :param Y: label values 
    :return: entropy cost
    """
    m = Y.shape[1]
    J = 1/m * np.dot(Y, np.log(AL.T)) + np.dot(1-Y, np.log(1-AL.T))
    J = np.squeeze(J)
    return J

### 4. 向后传播（Backward Propagation）
向后传播涉及一系列链式求导。中间L-1层的一系列梯度如下。 

激活函数的梯度

$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$

线性函数梯度
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$

而第L层的dAL如下：
$$dAL=\frac{\partial{J}}{\partial{A^{[L]}}}=-(\frac{Y}{A^{[L]}}-\frac{1-Y}{1-A^{[L]}})$$


#### 4.1 激活函数梯度
为了后面计算起来方便，先来写激活函数的梯度
**Relu**

In [ ]:
def relu_backward(dA, Z):
    dZ = np.array(dA, copy=True)
    dZ[Z<=0] = 0
    return dZ 


**sigmoid**

In [9]:
def sigmoid_backward(dA, Z):
    a = 1/(1+np.exp(-Z))
    dZ = dA * a * (1-a)
    return dZ

#### 4.2 线性好书梯度

In [14]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T, dZ)
    return dA_prev, dW, db

#### 4.3 一层网络中的backward
上面两步用到的函数连起来

In [23]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dZ, linear_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

#### 4.4 完整实现Backward Propagation
把上面的函数串起来，放到L层网络里面，完整实现BP

In [24]:
def backward_propagation(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # Initialization of dAL
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[-1]
    grads["A" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = \
        linear_activation_backward(dAL, current_cache,'sigmoid')
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = \
            linear_activation_backward(grads["dA"+str(l+2)], current_cache, 'relu')
        grads['dA' + str(l + 1)] = dA_prev_temp
        grads['dW' + str(l + 1)] = dW_temp
        grads['db' + str(l + 1)] = db_temp
    
    return grads

### 5. 梯度下降法更新权值

$W^{[l]}=W^{[l]}-\alpha{dW^{[l]}}$

$b^{[l]}=b^{[l]}-\alpha{db^{[l]}}$

In [25]:
def update_parameters(param_weights, grads, learning_rate):
    L = len(param_weights)
    for l in range(1, L+1):
        param_weights['W' + str(l)] -= learning_rate*grads['dW' + str(l)]
        param_weights['b' + str(l)] -= learning_rate*grads['db' + str(l)]
    return param_weights

In [26]:
### 6. 最终L层深度神经网络函数
def L_layer_model(X, Y, layers_dims_list, learning_rate = 0.001, num_iterations = 3000):
    np.random.seed(121)
    costs = []
    params_weights = l_layer_init(layers_dims_list)
    for i in range(num_iterations):
        AL, caches = forward_propagation(X, params_weights)
        costs.append(cost(AL, Y))
        grads = backward_propagation(AL, Y, caches)
        params_weights = update_parameters(params_weights, grads, learning_rate)
        if i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, costs[-1]))
    
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return params_weights

def predict(X, y, params_weight):
    m = X.shape[1]
    p = np.zeros([1, m])
    prob, _ = forward_propagation(X, params_weight)
    
    for i in range(0, prob.shape[1]):
        if prob[0, i]> 0.5:
            p[0, i] = 1
        else:
            p[0, i] = 0
    print("Accuracy: " + str(np.sum((p==y)/m)))
    return p
        

### 7. 导入数据进行实验
#### 7.1 导入h5格式的数据

In [4]:
import h5py
## load train dataset
train_dataset = h5py.File('datasets/train_catvnoncat.h5', 'r')
# train_set_x_raw = np.array(train_dataset["train_set_X"][:])
# train_set_y_raw = np.array(train_dataset["train_set_y"][:])
# ## load test dataset
# test_dataset = h5py.File('datasets/test_catvnoncat.h5', 'r')
# test_set_x_raw = np.array(train_dataset["test_set_X"][:])
# test_set_y_raw = np.array(train_dataset["test_set_y"][:])
# clases = np.array(test_dataset["list_classes"][:])
# 
# train_set_y_raw = train_set_y_raw.reshape((1, train_set_y_raw.shape[0]))
# test_set_y_raw = test_set_y_raw.reshape((1, test_set_y_raw.shape[0]))
# print(train_set_y_raw)


OSError: Unable to open file (File signature not found)

In [3]:
%pwd

'/Users/MIAOKUI/Documents/机器学习/LearningAI/CatRecognition'